In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from gensim import corpora, models, similarities, matutils
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

# Limit rows disp# Limit rows displayed in notebook
pd.set_option('display.max_rows', 15)
pd.set_option('display.precision', 2)

In [76]:
# run this cell once
from sklearn.cross_validation import train_test_split

rest_reviews = pd.read_pickle('review/reviews300k.pkl')


# get restaurant reviews
#train_data, test_data = train_test_split(rest_reviews, test_size=0.3)


In [77]:
rest_reviews.shape


(295005, 15)

In [4]:
# deserialize train_data and test_data
train_data = pd.read_pickle('review/train_data_all.pkl')


In [78]:
s1 = rest_reviews[0:20000]
s2 = rest_reviews[20000:40000]
s3 = rest_reviews[40000:60000]
s4 = rest_reviews[60000:80000]
s5 = rest_reviews[80000:100000]
s6 = rest_reviews[100000:120000]
s7 = rest_reviews[120000:140000]
s8 = rest_reviews[140000:160000]
s9 = rest_reviews[160000:180000]
s10 = rest_reviews[180000:200000]
s11 = rest_reviews[200000:220000]
s12 = rest_reviews[220000:240000]
s13 = rest_reviews[240000:260000]
s14 = rest_reviews[260000:28000]
s15 = rest_reviews[280000:]

In [216]:
def df_info(df, transpose):
    total_rows = df.shape[0]
    total_cols = df.shape[1]
    nan_df = pd.DataFrame(df.count(), columns=["# NaNs's"])
    nan_df["# NaNs's"] = nan_df["# NaNs's"].apply(lambda row: total_rows - row)
    print "Number of rows: %i" % total_rows
    print "Number of columns: %i" % total_cols
    print "Columns with NaN's:"
    if nan_df[nan_df["# NaNs's"]>0].empty:
        print 'None'
        return
    if transpose:
        print nan_df[nan_df["# NaNs's"]>0].T
    else:
        print nan_df[nan_df["# NaNs's"]>0]
    return

    

In [217]:
df_info(train_data, False)


Number of rows: 7000
Number of columns: 12
Columns with NaN's:
None


## LDA Model

In [80]:
def update(d, model, val, vectorizer):
    #vectorizer = CountVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2))
    train_documents = d['text'].values
    train_matrix = vectorizer.transform(train_documents)
    corpus = matutils.Sparse2Corpus(train_matrix.T)
    
    corpora.MmCorpus.serialize('/tmp/corpus'+str(val)+'.mm', corpus)
    corpus = corpora.MmCorpus('/tmp/corpus'+str(val)+'.mm')
    model.update(corpus)
    model.save('/tmp/model' + str(val) + '.mm')
    return

In [79]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=1, stop_words='english', ngram_range=(1, 3))
train_documents = rest_reviews['text'].values
train_matrix = vectorizer.fit_transform(train_documents)

train_documents1 = s1['text'].values
train_matrix1 = vectorizer.fit_transform(train_documents1)
corpus = matutils.Sparse2Corpus(train_matrix1.T)
corpora.MmCorpus.serialize('/tmp/corpuss1.mm', corpus)
corpus = corpora.MmCorpus('/tmp/corpuss1.mm')

In [81]:
dictionary = vectorizer.vocabulary_
gensim_dict = {}
for key, val in dictionary.items():
    gensim_dict[val] = key

In [82]:
lda_model = models.ldamodel.LdaModel(corpus, id2word=gensim_dict, num_topics=50)

In [83]:
update(s2, lda_model, 2, vectorizer)

In [84]:
update(s3, lda_model, 3, vectorizer)

In [85]:
update(s4, lda_model, 4, vectorizer)

In [86]:
update(s5, lda_model, 5, vectorizer)

In [87]:
update(s6, lda_model, 6, vectorizer)

In [88]:
update(s7, lda_model, 7, vectorizer)

In [89]:
update(s8, lda_model, 8, vectorizer)

In [90]:
update(s9, lda_model, 9, vectorizer)

In [91]:
update(s10, lda_model, 10, vectorizer)

In [92]:
update(s11, lda_model, 11, vectorizer)

In [93]:
update(s12, lda_model, 12, vectorizer)

In [94]:
update(s13, lda_model, 13, vectorizer)

In [95]:
update(s14, lda_model, 14, vectorizer)

In [96]:
update(s15, lda_model, 15, vectorizer)

In [97]:
lda_model.save('/tmp/model_online_all.lda')

In [103]:
#df = pd.DataFrame(lda_model.show_topics(num_topics=50, formatted=False))
lda_model.show_topics(num_topics=50, formatted=False)

[[(0.006583012686317993, u'omelette'),
  (0.0061953354933830512, u'short ribs'),
  (0.006171436770857042, u'close home'),
  (0.0059974434043739908, u'short rib'),
  (0.0049552684844034311, u'chairs'),
  (0.0049384971982386729, u'winner'),
  (0.0048561478085350808, u'verde'),
  (0.0047250111745640606, u'nice atmosphere'),
  (0.0044851711733215867, u'sample'),
  (0.0043765651853776592, u'stumbled')],
 [(0.048546333325605436, u'great'),
  (0.028770606827003959, u'place'),
  (0.021589772985277911, u'bar'),
  (0.019171966381187868, u'happy'),
  (0.016173249603160904, u'beer'),
  (0.014127621617055358, u'hour'),
  (0.012945611548294619, u'love'),
  (0.012105826939324158, u'staff'),
  (0.011422064809108744, u'happy hour'),
  (0.010278794353741759, u'drinks')],
 [(0.034190307368842009, u'pho'),
  (0.013212076609682427, u'cajun'),
  (0.0051053979415614809, u'glendale'),
  (0.0047558652665468503, u'dan'),
  (0.0045451149612565286, u'place opened'),
  (0.0045090941124227603, u'pickles'),
  (0.003

In [106]:
df = pd.DataFrame([[(0.006583012686317993, u'omelette'),
  (0.0061953354933830512, u'short ribs'),
  (0.006171436770857042, u'close home'),
  (0.0059974434043739908, u'short rib'),
  (0.0049552684844034311, u'chairs'),
  (0.0049384971982386729, u'winner'),
  (0.0048561478085350808, u'verde'),
  (0.0047250111745640606, u'nice atmosphere'),
  (0.0044851711733215867, u'sample'),
  (0.0043765651853776592, u'stumbled')],
 [(0.048546333325605436, u'great'),
  (0.028770606827003959, u'place'),
  (0.021589772985277911, u'bar'),
  (0.019171966381187868, u'happy'),
  (0.016173249603160904, u'beer'),
  (0.014127621617055358, u'hour'),
  (0.012945611548294619, u'love'),
  (0.012105826939324158, u'staff'),
  (0.011422064809108744, u'happy hour'),
  (0.010278794353741759, u'drinks')],
 [(0.034190307368842009, u'pho'),
  (0.013212076609682427, u'cajun'),
  (0.0051053979415614809, u'glendale'),
  (0.0047558652665468503, u'dan'),
  (0.0045451149612565286, u'place opened'),
  (0.0045090941124227603, u'pickles'),
  (0.0037969057944371633, u'recommendations'),
  (0.0034926752963152629, u'service place'),
  (0.0031939747154203723, u'server great'),
  (0.0030901638382888541, u'mint')],
 [(0.028159464154650163, u'beans'),
  (0.015643753793493968, u'taco'),
  (0.011007184851579465, u'burrito'),
  (0.010233757685706755, u'chipotle'),
  (0.0083094489655714115, u'bean'),
  (0.0070210354849193045, u'black'),
  (0.005900025243739844, u'support'),
  (0.0058772166662999573, u'pastor'),
  (0.005700458634719051, u'shredded'),
  (0.0050256303087835279, u'black beans')],
 [(0.010119071446204431, u'jalapeno'),
  (0.0098368840404260919, u'breakfast lunch'),
  (0.0098036961790226602, u'belt'),
  (0.0090989938759820053, u'zucchini'),
  (0.0069683446877427356, u'subs'),
  (0.0060164773450787313, u'corned'),
  (0.0059429312392616073, u'corned beef'),
  (0.0047067110585962543, u'pies'),
  (0.0043915742873611441, u'iced coffee'),
  (0.0042249591047481914, u'opens')],
 [(0.038956459939547362, u'free'),
  (0.024110588304085433, u'gluten'),
  (0.020944168894817414, u'gluten free'),
  (0.014022244880786848, u'chinese'),
  (0.010350619067221874, u'new place'),
  (0.0061296456058625619, u'grand opening'),
  (0.0059921636029424625, u'chinese food'),
  (0.0049192513700467263, u'scottsdale'),
  (0.0047520165059032121, u'gf'),
  (0.0042986980705017226, u'north scottsdale')],
 [(0.045861456846669493, u'chicken'),
  (0.022182238223521278, u'rice'),
  (0.019284988366951993, u'sauce'),
  (0.017816441865047689, u'fried'),
  (0.017080423803157777, u'soup'),
  (0.015931264103893006, u'spicy'),
  (0.014600555581534039, u'shrimp'),
  (0.013636371228079609, u'ordered'),
  (0.011433677783038139, u'flavor'),
  (0.011196917934108574, u'dish')],
 [(0.037403805236083984, u'good'),
  (0.036071956832650937, u'food'),
  (0.024067074271675041, u'place'),
  (0.019311624505164545, u'great'),
  (0.015969141859501414, u'service'),
  (0.013540960006822911, u'really'),
  (0.0094847197998976906, u've'),
  (0.0090083143289782022, u'like'),
  (0.0089972159420659299, u'delicious'),
  (0.0087540229274007449, u'restaurant')],
 [(0.01954689115531643, u'italian'),
  (0.0058849679231436705, u'amazing service'),
  (0.0054012482799018543, u'great new'),
  (0.0052972640345168454, u'pasta'),
  (0.0052493161280768958, u'try menu'),
  (0.0050990512102355333, u'date night'),
  (0.0049695668664059382, u'extremely'),
  (0.0046495840460493093, u'garden'),
  (0.0045118326164467638, u'flatbread'),
  (0.0043675037574102144, u'extremely friendly')],
 [(0.013632983248451911, u'pretzel'),
  (0.007429859339664952, u'great pizza'),
  (0.0073621029539848908, u'buds'),
  (0.006932825142364798, u'taste buds'),
  (0.0062385757943077078, u'cauliflower'),
  (0.0055375143766430513, u'awesome service'),
  (0.0046888189204421218, u'good size'),
  (0.0045684751917847355, u'rave'),
  (0.0043609249918794462, u'little miss'),
  (0.0043083000248900815, u'sat patio')],
 [(0.04584915823185829, u'thai'),
  (0.01192656067643586, u'pad'),
  (0.0097081089657284642, u'pad thai'),
  (0.0091197750969819226, u'outdoor'),
  (0.0081281452939714043, u'seating'),
  (0.0080667312106026366, u'coconut'),
  (0.0077703103119121306, u'ramen'),
  (0.0069403940318165623, u'staff super'),
  (0.0068707782580339936, u'thai food'),
  (0.0058241732569306141, u'outdoor seating')],
 [(0.011676115238295195, u'concept'),
  (0.0063816699868959438, u'pricey'),
  (0.0055682559249265514, u'chef'),
  (0.0054218625207042275, u'fountain'),
  (0.0052681564966131885, u'organic'),
  (0.0049100636706661911, u'welcome'),
  (0.0049088826782229637, u'little pricey'),
  (0.0044979412919483215, u'smoked'),
  (0.0044674674571028002, u'mike'),
  (0.0043435353856457427, u'selections')],
 [(0.020453992549401716, u'excellent'),
  (0.015522756644916142, u'food excellent'),
  (0.011999275078811041, u'vietnamese'),
  (0.011446616464342446, u'excellent service'),
  (0.0072424218739075546, u'pizza good'),
  (0.00625164183407915, u'good atmosphere'),
  (0.0059047864765407491, u'spacious'),
  (0.0049163466569311061, u'giving stars'),
  (0.0044420077698681913, u'chris'),
  (0.0042694254871241859, u'good happy')],
 [(0.067318061932947773, u'breakfast'),
  (0.046571758749852303, u'coffee'),
  (0.022665578901947422, u'eggs'),
  (0.016648426183920491, u'bacon'),
  (0.016284868573109235, u'brunch'),
  (0.012019774412705842, u'pancakes'),
  (0.010018751634775587, u'egg'),
  (0.0076046688540061672, u'gravy'),
  (0.0075798662739035259, u'sunday'),
  (0.0072138703562187774, u'new restaurant')],
 [(0.022345939825133204, u'pie'),
  (0.012396325672960571, u'customer'),
  (0.012245755689981, u'customer service'),
  (0.0094378373200990264, u'garlic'),
  (0.0079607786601861123, u'slice'),
  (0.0074051461991008292, u'definitely coming'),
  (0.0058838324555246626, u'great customer'),
  (0.0057646537684110412, u'great customer service'),
  (0.0051298932787703592, u'croissant'),
  (0.0045392070781272893, u'ny')],
 [(0.0078292066655310522, u'feels'),
  (0.0072473392096462901, u'west'),
  (0.0065770679975943437, u'asu'),
  (0.0063633802773839085, u'shopping'),
  (0.0061914767198593592, u'far best'),
  (0.0058032277950025128, u'angry'),
  (0.0053333218730767917, u'great price'),
  (0.0047290276370692058, u'center'),
  (0.0044091398565826957, u'feels like'),
  (0.0044055802880024822, u'goods')],
 [(0.036941114297974767, u'sandwich'),
  (0.032662336238087911, u'salad'),
  (0.022750242937667718, u'cheese'),
  (0.017372737359601407, u'bread'),
  (0.017150178518282433, u'bbq'),
  (0.015208261574842986, u'sauce'),
  (0.01290755212076713, u'pork'),
  (0.012884051004193929, u'meat'),
  (0.012852628597236652, u'sandwiches'),
  (0.011414907894491084, u'chicken')],
 [(0.02622457995681414, u'highly'),
  (0.025435506988366229, u'recommend'),
  (0.024080652167799286, u'highly recommend'),
  (0.010135889251920917, u'recommend place'),
  (0.0070682450198778701, u'medium'),
  (0.0068741633984551629, u'absolutely'),
  (0.0066372096418825955, u'rare'),
  (0.0057563552744207781, u'knowledgeable'),
  (0.005675096225310032, u'filet'),
  (0.0053105470060693357, u'king')],
 [(0.018117571228842069, u'wings'),
  (0.0090384490354591396, u'iced'),
  (0.0084003280078979101, u'tea'),
  (0.0065879132264356141, u'asian'),
  (0.0061158420762339712, u'iced tea'),
  (0.0047316244698761408, u'quinoa'),
  (0.0046452127598110649, u'beer selection'),
  (0.0046432764674936948, u'pot'),
  (0.0045383619386316423, u'boba'),
  (0.0037794663696883658, u'milk')],
 [(0.0044558472071483122, u'clams'),
  (0.0040762384040587008, u'1st'),
  (0.0038096330175989091, u'sam'),
  (0.0035440825176304282, u'bc'),
  (0.0034562793475676704, u'fiance'),
  (0.0033945298721350142, u'difference'),
  (0.0033216164065706147, u'people working'),
  (0.003278913490547659, u'sriracha'),
  (0.0031116531884343179, u'healthy food'),
  (0.0030044114357678183, u'dancing')],
 [(0.068882439303848034, u'sushi'),
  (0.03604882363402305, u'roll'),
  (0.023513465420264855, u'rolls'),
  (0.011304589586856239, u'tuna'),
  (0.0093935788290457328, u'salmon'),
  (0.0085896438635003226, u'chef'),
  (0.0076750759723011788, u'fish'),
  (0.0068071993590627031, u'japanese'),
  (0.0059081387366385742, u'fresh'),
  (0.0056202374458250931, u'truck')],
 [(0.015540871906365276, u'food'),
  (0.014671495781674228, u'time'),
  (0.01220109936512967, u'just'),
  (0.010811614980838952, u'order'),
  (0.01027593229837683, u'ordered'),
  (0.0094966572920363781, u'came'),
  (0.0094743091214417145, u'like'),
  (0.0091104253475274479, u'got'),
  (0.0088526916622375714, u'service'),
  (0.0083499217317287554, u'place')],
 [(0.04172858599589227, u'ice'),
  (0.031347263825134537, u'cream'),
  (0.027700080356823812, u'chocolate'),
  (0.019333359031340001, u'ice cream'),
  (0.017030809507263836, u'cake'),
  (0.012038515228031023, u'dessert'),
  (0.011081683040348404, u'apple'),
  (0.0075496128914469014, u'cookie'),
  (0.0062476172939412919, u'vanilla'),
  (0.0061769155652834848, u'milk')],
 [(0.064190286387063353, u'tacos'),
  (0.028795444746035934, u'burrito'),
  (0.024258617321646445, u'taco'),
  (0.020425262760129782, u'carne'),
  (0.020138377855458949, u'asada'),
  (0.017963627930503802, u'carne asada'),
  (0.012013566321198814, u'guacamole'),
  (0.0099781153959446062, u'tortillas'),
  (0.0088965007014884592, u'corn'),
  (0.0086740830780552072, u'burritos')],
 [(0.021724397100755844, u'vegan'),
  (0.015423090934128693, u'old town'),
  (0.012912891965636332, u'shake'),
  (0.0088405446684289846, u'valet'),
  (0.0083926105863850829, u'food really good'),
  (0.0061783204007025989, u'jungle'),
  (0.0050334298630688636, u'locally'),
  (0.0049472819439371502, u'alternative'),
  (0.0049024756925835564, u'terminal'),
  (0.0047898097077268848, u'catfish')],
 [(0.013295172926332907, u'juice'),
  (0.013219191035815528, u'potato fries'),
  (0.013000322864228704, u'sweet potato fries'),
  (0.012943976843256691, u'fox'),
  (0.012731329992910588, u'sweet potato'),
  (0.0083542648546324726, u'perfection'),
  (0.0082419485648275344, u'juices'),
  (0.0079063074598117351, u'cooked perfection'),
  (0.0058667305320402804, u'smashburger'),
  (0.0057629765130790024, u'native')],
 [(0.011578133379651181, u'hash'),
  (0.0082960591165221319, u'grits'),
  (0.007332189843152875, u'trifecta'),
  (0.0065806525693728381, u'mesa'),
  (0.0061705158780674318, u'browns'),
  (0.0058611099365137075, u'flight'),
  (0.0056569842763447356, u'hash browns'),
  (0.0042799261927545839, u'smoothies'),
  (0.0038356170600575006, u'new spot'),
  (0.0037522760958681395, u'key')],
 [(0.024331598183812232, u'delivery'),
  (0.0083453309972500445, u'snow'),
  (0.0076625892932432778, u'vegas'),
  (0.0066974238478491599, u'brat'),
  (0.0052662459446469233, u'dj'),
  (0.0050738635526614588, u'drunken'),
  (0.0048153665459258685, u'chip'),
  (0.0046514961156547191, u'details'),
  (0.0041829433974239312, u'rolled'),
  (0.0040853637368116567, u'placed order')],
 [(0.015012504283731591, u'greek'),
  (0.012910500872515232, u'gyro'),
  (0.010424611322237223, u'bloody'),
  (0.0099492388599478031, u'biscuits'),
  (0.0098006358446756058, u'trendy'),
  (0.009422613894733296, u'mary'),
  (0.0085801257982444715, u'bloody mary'),
  (0.0079027981271081842, u'concept'),
  (0.0076122695952212513, u'craft'),
  (0.0045857039748450484, u'biscuits gravy')],
 [(0.0081352226495885766, u'bay'),
  (0.0066568654326349605, u'scottsdale'),
  (0.0054828964618114989, u'don forget'),
  (0.0052293766444345144, u'forget'),
  (0.0051362762965014158, u'san'),
  (0.0044381999988508782, u'town scottsdale'),
  (0.0043503354842293947, u'old town scottsdale'),
  (0.0043305184388765829, u'new place'),
  (0.0042619334173730544, u'pecan'),
  (0.0041855892869318186, u'taken care')],
 [(0.015394642754588047, u'mall'),
  (0.013465882395563256, u'strip'),
  (0.010657570938557686, u'golf'),
  (0.009477601524562099, u'hopes'),
  (0.0088580993741482587, u'strip mall'),
  (0.0073903393096273794, u'multiple'),
  (0.0071775489294249413, u'football'),
  (0.0069251494015128137, u'blown'),
  (0.0066583359867607049, u'love food'),
  (0.0058946471374905279, u'high hopes')],
 [(0.01189204639700221, u'waffles'),
  (0.010257872816148437, u'ahi'),
  (0.0078772592445344198, u'awesome food'),
  (0.0077901678865059778, u'omelet'),
  (0.0073849661466296873, u'smoothie'),
  (0.0071873672405504772, u'service awesome'),
  (0.0066363595446206234, u'horchata'),
  (0.0062768575764507858, u'chicken waffles'),
  (0.0056163003646695336, u'conveyor belt'),
  (0.0054608809408858471, u'food fresh')],
 [(0.028118170522112827, u'buffet'),
  (0.024683048422028817, u'indian'),
  (0.013569418028958821, u'donut'),
  (0.011622305895110105, u'naan'),
  (0.010829067669608401, u'tempe'),
  (0.0092799139476528077, u'indian food'),
  (0.008435021864019062, u'masala'),
  (0.0068993263822342606, u'lunch buffet'),
  (0.0067634321138169227, u'tikka'),
  (0.0059824640458862989, u'legs')],
 [(0.016210778888245424, u'mac'),
  (0.015354195431747191, u'mac cheese'),
  (0.013789820577962608, u'perfectly'),
  (0.011304736599854407, u'cooked'),
  (0.010720249710212187, u'dessert'),
  (0.0097525394421291208, u'kale'),
  (0.0088224687393068223, u'lobster'),
  (0.0085930169609529511, u'salmon'),
  (0.008269864979364697, u'crab'),
  (0.0065953697982318619, u'course')],
 [(0.011010033654710163, u'gilbert'),
  (0.0081415900263900297, u'gem'),
  (0.0081025527270319423, u'hidden'),
  (0.0080744919820503764, u'best'),
  (0.007538884351676196, u'hands'),
  (0.0061034768183107655, u'central'),
  (0.0057102436947062277, u'hidden gem'),
  (0.005215425507382773, u'friendly attentive'),
  (0.0048674871258383061, u'month'),
  (0.0047519120675179665, u'postino')],
 [(0.13935650638566691, u'pizza'),
  (0.024166714262187094, u'crust'),
  (0.014092365900226479, u'chicago'),
  (0.0127120694645658, u'toppings'),
  (0.012342655611614734, u'pizzas'),
  (0.0079939863865394131, u'style'),
  (0.0077035115531531558, u'new'),
  (0.0068329879150783842, u'sausage'),
  (0.0061728539151048355, u'sauce'),
  (0.0060640933085519952, u'slices')],
 [(0.0076973436550487355, u'closed'),
  (0.0057864602123354958, u'really wanted'),
  (0.0057651768852176174, u'standing'),
  (0.005683999954423577, u'oysters'),
  (0.0054271560299907276, u'deliver'),
  (0.0053124738878418147, u'online'),
  (0.0052907615162627103, u'sad'),
  (0.0043670049589727986, u'takeout'),
  (0.0039216190498757202, u'wanted like'),
  (0.003847946030925046, u'tonight')],
 [(0.025831611380089482, u'hummus'),
  (0.023327965160434685, u'pita'),
  (0.0086774178312446926, u'mediterranean'),
  (0.0074034718220568393, u'fell'),
  (0.0048024488835624228, u'good burger'),
  (0.0045718792916993895, u'asian'),
  (0.0042579577323420383, u'slow service'),
  (0.0039856630174320085, u'new place'),
  (0.00368894726383533, u'took bite'),
  (0.0033143792311670322, u'koi')],
 [(0.0078973345728463144, u'bread'),
  (0.0076742815752926822, u'pudding'),
  (0.0075975960406973126, u'oil'),
  (0.0070866733623022567, u'pesto'),
  (0.0060150610417883926, u'tempe'),
  (0.0056131969911206838, u'tomatoes'),
  (0.0050493400549055491, u'olive'),
  (0.0049383265139463358, u'greens'),
  (0.0048021729569064191, u'mozzarella'),
  (0.0046139644386256609, u'veggie')],
 [(0.02023227091381857, u'burger'),
  (0.01515235080258997, u'fries'),
  (0.013510612336940054, u'like'),
  (0.011764363999092387, u'just'),
  (0.0080392050726867843, u'cheese'),
  (0.0079956771792321548, u'hot'),
  (0.0063653794761531022, u'don'),
  (0.0060697004753637986, u'burgers'),
  (0.005836197938951378, u'scottsdale'),
  (0.0042489945086277075, u'make')],
 [(0.012296283350070054, u'love love'),
  (0.0070198392698392837, u'fish chips'),
  (0.0062534157720221971, u'muffin'),
  (0.0054824142498884022, u'love love love'),
  (0.0048845431038663659, u'months ago'),
  (0.0047500120353361815, u'blueberry'),
  (0.0039644633255663123, u'bike'),
  (0.0038215546511630493, u'cod'),
  (0.0035796879312393975, u'stone'),
  (0.0035671951516449553, u'cobb')],
 [(0.01375514351570315, u'rib'),
  (0.0069486409245967324, u'lo'),
  (0.0065014926035582427, u'fried'),
  (0.0061953913528204219, u'waffle'),
  (0.0059906824916846034, u'prime'),
  (0.0051822078044595635, u'latte'),
  (0.005047402971944003, u'sashimi'),
  (0.0045731765342823015, u'melt'),
  (0.0040263362967405611, u'prime rib'),
  (0.0034454025065063063, u'place new')],
 [(0.0062511805050892502, u'yelp'),
  (0.0060765486283472486, u'jimmy'),
  (0.0059926542941934093, u'com'),
  (0.0056869405483999319, u'chopped'),
  (0.0056179687106045805, u'distance'),
  (0.0050964569329238485, u'torta'),
  (0.0049440938366631321, u'great addition'),
  (0.0047178301233252555, u'arcadia'),
  (0.0047085927674399503, u'screen'),
  (0.0045084038463702241, u'select')],
 [(0.025965463069377581, u'toast'),
  (0.018194710657568704, u'onion'),
  (0.01548215916309904, u'rings'),
  (0.014475256579987973, u'french'),
  (0.013500009729832285, u'onion rings'),
  (0.013110794871001974, u'french toast'),
  (0.0077614779843836112, u'paradise'),
  (0.0056352288802162575, u'pink'),
  (0.0051016053382041959, u'tenders'),
  (0.0050642639407088514, u'bakery')],
 [(0.023294607389279284, u'wine'),
  (0.0094254736171684225, u'list'),
  (0.008252511081034206, u'reservation'),
  (0.0076899390274739621, u'just opened'),
  (0.0055052093645404296, u'yum'),
  (0.0050446168918932138, u'wine list'),
  (0.0046558083692765885, u'bother'),
  (0.0045945884543144438, u'lamb'),
  (0.0035447379467340727, u'caesar salad'),
  (0.0034620819060883368, u'explained')],
 [(0.040754969738426794, u'steak'),
  (0.020694265163138405, u'potatoes'),
  (0.0082331122984385177, u'mashed'),
  (0.0065942832300326477, u'mashed potatoes'),
  (0.0064767008609303966, u'medium'),
  (0.0064624084415136548, u'value'),
  (0.0062824399507647437, u'beers tap'),
  (0.0044751277855690176, u'craft'),
  (0.0043698220068183382, u'hour menu'),
  (0.0043106650556252023, u'whiskey')],
 [(0.045211122764714583, u'great'),
  (0.026202498864895382, u'great food'),
  (0.024876026091981064, u'food great'),
  (0.021005462120476628, u'great service'),
  (0.01239706769260991, u'service great'),
  (0.012147104614368767, u'crawfish'),
  (0.010632714343504502, u'kinks'),
  (0.010584659391889401, u'food great service'),
  (0.010020071196745599, u'crepe'),
  (0.0093219956582255275, u'great food great')],
 [(0.010742496577928456, u'italian beef'),
  (0.0067711411970974586, u'artichoke'),
  (0.004901639585449551, u'came quickly'),
  (0.0047199012374386957, u'did great'),
  (0.0040973606424184045, u'german'),
  (0.0038151145139299378, u'condensed'),
  (0.0035771924310304369, u'battered'),
  (0.0035446891425379764, u'grass'),
  (0.0035204051696305291, u'did great job'),
  (0.0034101554957746555, u'second chance')],
 [(0.0075377113810093845, u'wait come'),
  (0.0056543144625131559, u'macaroni'),
  (0.0055999101195315804, u'matt'),
  (0.0052454069514299879, u'cheesecake'),
  (0.0046027569302327879, u'pastry'),
  (0.0040365961524122043, u'menu food'),
  (0.0039456184326418465, u'good restaurant'),
  (0.0038270455170934482, u'whipped'),
  (0.0037653414133662611, u'came time'),
  (0.0035078389109082968, u'experience great')],
 [(0.021193777646127093, u'forward'),
  (0.012110231738382063, u'looking forward'),
  (0.0074478889890914604, u'right away'),
  (0.0070789922334004492, u'look forward'),
  (0.0045562246105853724, u'charged'),
  (0.0037764121217397049, u'apps'),
  (0.0036987396464963945, u'middle'),
  (0.003576834151454958, u'forward trying'),
  (0.0031335479852677662, u'energy'),
  (0.0029666960041394956, u'looking')]])

In [107]:
df

,0,1,2,3,4,5,6,7,8,9
0,"(0.00658301268632, omelette)","(0.00619533549338, short ribs)","(0.00617143677086, close home)","(0.00599744340437, short rib)","(0.0049552684844, chairs)","(0.00493849719824, winner)","(0.00485614780854, verde)","(0.00472501117456, nice atmosphere)","(0.00448517117332, sample)","(0.00437656518538, stumbled)"
1,"(0.0485463333256, great)","(0.028770606827, place)","(0.0215897729853, bar)","(0.0191719663812, happy)","(0.0161732496032, beer)","(0.0141276216171, hour)","(0.0129456115483, love)","(0.0121058269393, staff)","(0.0114220648091, happy hour)","(0.0102787943537, drinks)"
2,"(0.0341903073688, pho)","(0.0132120766097, cajun)","(0.00510539794156, glendale)","(0.00475586526655, dan)","(0.00454511496126, place opened)","(0.00450909411242, pickles)","(0.00379690579444, recommendations)","(0.00349267529632, service place)","(0.00319397471542, server great)","(0.00309016383829, mint)"
3,"(0.0281594641547, beans)","(0.0156437537935, taco)","(0.0110071848516, burrito)","(0.0102337576857, chipotle)","(0.00830944896557, bean)","(0.00702103548492, black)","(0.00590002524374, support)","(0.0058772166663, pastor)","(0.00570045863472, shredded)","(0.00502563030878, black beans)"
4,"(0.0101190714462, jalapeno)","(0.00983688404043, breakfast lunch)","(0.00980369617902, belt)","(0.00909899387598, zucchini)","(0.00696834468774, subs)","(0.00601647734508, corned)","(0.00594293123926, corned beef)","(0.0047067110586, pies)","(0.00439157428736, iced coffee)","(0.00422495910475, opens)"
5,"(0.0389564599395, free)","(0.0241105883041, gluten)","(0.0209441688948, gluten free)","(0.0140222448808, chinese)","(0.0103506190672, new place)","(0.00612964560586, grand opening)","(0.00599216360294, chinese food)","(0.00491925137005, scottsdale)","(0.0047520165059, gf)","(0.0042986980705, north scottsdale)"
6,"(0.0458614568467, chicken)","(0.0221822382235, rice)","(0.019284988367, sauce)","(0.017816441865, fried)","(0.0170804238032, soup)","(0.0159312641039, spicy)","(0.0146005555815, shrimp)","(0.0136363712281, ordered)","(0.011433677783, flavor)","(0.0111969179341, dish)"
...,...,...,...,...,...,...,...,...,...,...
43,"(0.0259654630694, toast)","(0.0181947106576, onion)","(0.0154821591631, rings)","(0.01447525658, french)","(0.0135000097298, onion rings)","(0.013110794871, french toast)","(0.00776147798438, paradise)","(0.00563522888022, pink)","(0.0051016053382, tenders)","(0.00506426394071, bakery)"
44,"(0.0232946073893, wine)","(0.00942547361717, list)","(0.00825251108103, reservation)","(0.00768993902747, just opened)","(0.00550520936454, yum)","(0.00504461689189, wine list)","(0.00465580836928, bother)","(0.00459458845431, lamb)","(0.00354473794673, caesar salad)","(0.00346208190609, explained)"


In [108]:
df = df.applymap(lambda x: x[1])
lda_columns = map( lambda x: 'word_' + str(x), df.columns.values)
df.columns = lda_columns
df['words'] = map(list, df.values)
df.head()

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,words
0,omelette,short ribs,close home,short rib,chairs,winner,verde,nice atmosphere,sample,stumbled,"[omelette, short ribs, close home, short rib, ..."
1,great,place,bar,happy,beer,hour,love,staff,happy hour,drinks,"[great, place, bar, happy, beer, hour, love, s..."
2,pho,cajun,glendale,dan,place opened,pickles,recommendations,service place,server great,mint,"[pho, cajun, glendale, dan, place opened, pick..."
3,beans,taco,burrito,chipotle,bean,black,support,pastor,shredded,black beans,"[beans, taco, burrito, chipotle, bean, black, ..."
4,jalapeno,breakfast lunch,belt,zucchini,subs,corned,corned beef,pies,iced coffee,opens,"[jalapeno, breakfast lunch, belt, zucchini, su..."


In [109]:
k = 0
topic_dict = {}
for val in df['words']:
    print 'Topic ' + str(k) + ':'
    print val
    topic_dict[k] = val
    k+=1


Topic 0:
[u'omelette', u'short ribs', u'close home', u'short rib', u'chairs', u'winner', u'verde', u'nice atmosphere', u'sample', u'stumbled']
Topic 1:
[u'great', u'place', u'bar', u'happy', u'beer', u'hour', u'love', u'staff', u'happy hour', u'drinks']
Topic 2:
[u'pho', u'cajun', u'glendale', u'dan', u'place opened', u'pickles', u'recommendations', u'service place', u'server great', u'mint']
Topic 3:
[u'beans', u'taco', u'burrito', u'chipotle', u'bean', u'black', u'support', u'pastor', u'shredded', u'black beans']
Topic 4:
[u'jalapeno', u'breakfast lunch', u'belt', u'zucchini', u'subs', u'corned', u'corned beef', u'pies', u'iced coffee', u'opens']
Topic 5:
[u'free', u'gluten', u'gluten free', u'chinese', u'new place', u'grand opening', u'chinese food', u'scottsdale', u'gf', u'north scottsdale']
Topic 6:
[u'chicken', u'rice', u'sauce', u'fried', u'soup', u'spicy', u'shrimp', u'ordered', u'flavor', u'dish']
Topic 7:
[u'good', u'food', u'place', u'great', u'service', u'really', u've', u'

### Get the topics and topic probability distributions for our entire review data set

In [123]:
def get_topics(model, corpus):
    """ Takes a corpus and finds the topic id and word distribution of each document in the corpus.
        It returns a list of the topic ids and word distributions. """
    topic_id = []
    topic_prob = []
    word_dist = []
    corpus = list(corpus)
    for doc in corpus:
        topics = model[doc]
        topics_sorted = list(sorted(topics, key=lambda x:x[1]))
        
        if len(topics_sorted) >=3:
            top_ids = {'topic1':topics_sorted[-1][0], 'topic2':topics_sorted[-2][0], 'topic3': topics_sorted[-3][0]}
            probs = {'topic_prob1':topics_sorted[-1][1], 'topic_prob2':topics_sorted[-2][1], 'topic_prob3':topics_sorted[-3][1]}
            dist = {'word_dist1':model.show_topic(topics_sorted[-1][0]), 'word_dist2':model.show_topic(topics_sorted[-2][0]), 'word_dist3':model.show_topic(topics_sorted[-3][0])}
        elif len(topics_sorted) >=2:
            top_ids = {'topic1':topics_sorted[-1][0], 'topic2':topics_sorted[-2][0]}
            probs = {'topic_prob1':topics_sorted[-1][1], 'topic_prob2':topics_sorted[-2][1]}
            dist = {'word_dist1':model.show_topic(topics_sorted[-1][0]), 'word_dist2':model.show_topic(topics_sorted[-2][0])}
        else:
            top_ids = {'topic1':topics_sorted[-1][0]}
            probs = {'topic_prob1':topics_sorted[-1][1]}
            dist = {'word_dist1':model.show_topic(topics_sorted[-1][0])}
        topic_id.append(top_ids) # most related topic id
        topic_prob.append(probs)
        word_dist.append(dist)
    return topic_id, topic_prob, word_dist

In [121]:
corpus1 = corpora.MmCorpus('/tmp/corpuss1.mm')
corpus2 = corpora.MmCorpus('/tmp/corpus2.mm')
corpus3 = corpora.MmCorpus('/tmp/corpus3.mm')
corpus4 = corpora.MmCorpus('/tmp/corpus4.mm')
corpus5 = corpora.MmCorpus('/tmp/corpus5.mm')
corpus6 = corpora.MmCorpus('/tmp/corpus6.mm')
corpus7 = corpora.MmCorpus('/tmp/corpus7.mm')
corpus8 = corpora.MmCorpus('/tmp/corpus8.mm')
corpus9 = corpora.MmCorpus('/tmp/corpus9.mm')
corpus10 = corpora.MmCorpus('/tmp/corpus10.mm')
corpus11 = corpora.MmCorpus('/tmp/corpus11.mm')
corpus12 = corpora.MmCorpus('/tmp/corpus12.mm')
corpus13 = corpora.MmCorpus('/tmp/corpus13.mm')
corpus14 = corpora.MmCorpus('/tmp/corpus14.mm')
corpus15 = corpora.MmCorpus('/tmp/corpus15.mm')


In [124]:
t_id1, t_prob1, w_dist1 = get_topics(lda_model, corpus1)

KeyboardInterrupt: 

In [119]:
topic_id = []
topic_prob = []
word_dist = []
corpus1 = list(corpus1)
for doc in corpus1:
    topics = lda_model[doc]
    topics_sorted = list(sorted(topics, key=lambda x:x[1]))
        
    top_ids = {'topic1':topics_sorted[-1][0], 'topic2':topics_sorted[-2][0], 'topic3': topics_sorted[-3][0]}
    probs = {'topic_prob1':topics_sorted[-1][1], 'topic_prob2':topics_sorted[-2][1], 'topic_prob3':topics_sorted[-3][1]}
    dist = {'word_dist1':lda_model.show_topic(topics_sorted[-1][0]), 'word_dist2':lda_model.show_topic(topics_sorted[-2][0]), 'word_dist3':lda_model.show_topic(topics_sorted[-3][0])}
    topic_id.append(top_ids) # most related topic id
    topic_prob.append(probs)
    word_dist.append(dist)

KeyboardInterrupt: 

In [236]:
#serialize topic id/word distribution lists 
pd.DataFrame(test_topic_id).to_pickle('review/test_topic_id.pkl')
pd.DataFrame(test_topic_prob).to_pickle('review/test_topic_prob.pkl')
pd.DataFrame(test_word_dist).to_pickle('review/test_topic_dist.pkl')

In [237]:
test_data['topic_id_50'] = test_topic_id_50
test_data['topic_prob_50'] = test_topic_prob_50
test_data['word_dist_50'] = test_word_dist_50

test_data.to_pickle('review/train_data_topics.pkl')
test_data.head()

,business_id,review_id,date.year,date.month,date.day,stars,text,review_length,votes.cool,votes.funny,votes.useful,user_id,topic_id_50,topic_prob_50,word_dist_50
0,x_mxrJALBi5No9CxRswEaQ,KmZzQ5qPYxtSOZiS5rQJiQ,2011,5,19,3,This place is a little different than the over...,1557,1,0,1,p7dvahRPZIWL7T6pFA_0dg,39,0.2,"[(0.00977830773598, place), (0.00650128695031,..."
1,E4b5OC_6mZ0V7B6Nyjncsg,Jnt2z0W4hzc63qau4pCjIA,2010,11,23,4,4 1/2 stars - really great place! The decor i...,221,1,0,1,pEVf8GRshP9HUkSpizc9LA,3,0.2,"[(0.0153710636923, great), (0.0092647577608, p..."
2,3EAF9e45FH01odIIglru3g,_dpto8wcsvVEQQgTOX-YLQ,2013,10,4,5,The food was flavorful and the staff worked as...,93,0,0,0,4f6CdApcsdOPhcwSOmM4qw,46,0.3,"[(0.00681956996686, food), (0.00665234161418, ..."
3,_UOaoy1lcdIbgaA-d9dqpg,OpjIiDFhzi_kW_5veg_ZOw,2014,7,5,3,Mediocre sandwich place for sure. Place is cle...,205,0,0,0,grt5IJNerHdP4PMwc1cMdg,38,0.2,"[(0.0133049281147, good), (0.0124872546346, pl..."
4,LQGQ1w4pYrf5in6LrU0BlQ,TEJGrTBQX__79WwoBcJAnQ,2014,6,20,5,This place has been around for years and has t...,253,0,0,1,Ayx_wLGPjTVPIYoFerF82Q,48,0.6,"[(0.0146616328294, good), (0.0142812265826, fo..."


In [238]:
topic_id_50, topic_prob_50, word_dist_50 = get_topics(lda_model_50, corpus)

In [239]:
#serialize topic id/word distribution lists 
pd.DataFrame(topic_id_50).to_pickle('review/train_topic_id_50.pkl')
pd.DataFrame(topic_prob_50).to_pickle('review/train_topic_prob_50.pkl')
pd.DataFrame(word_dist_50).to_pickle('review/train_topic_dist_50.pkl')

In [240]:
train_data['topic_id_50'] = topic_id_50
train_data['topic_prob_50'] = topic_prob_50
train_data['word_dist_50'] = word_dist_50

train_data.to_pickle('review/train_data_topics.pkl')

train_data.head()


,business_id,review_id,date.year,date.month,date.day,stars,text,review_length,votes.cool,votes.funny,votes.useful,user_id,topic_id_50,topic_prob_50,word_dist_50
0,E7nk0lC317pGxMX56gvaVQ,onOLEBffcpWS_jcaBLCqdw,2008,6,17,1,This place is a huge waste of the space they m...,1239,3,4,2,eBFm-lABQiKpaUcPDfYOgA,39,0.6,"[(0.00977830773598, place), (0.00650128695031,..."
1,duHUQFn7K-Ybs1rPI4AOEQ,23z4mKkaMCXGFw4cm6xHxA,2012,4,15,3,I used to frequent Z-Tejas more frequently for...,936,1,1,2,y5_vrqIylhXMrejM_-x_vA,43,0.7,"[(0.0090865194881, good), (0.00526586990462, g..."
2,_K5cfCM7-zQZ1hXdbsc2fQ,dqrwXpcYdFvNYL5z1AeuZw,2013,12,1,3,"Meeting some friends for dinner, they live in ...",1119,0,0,0,8si1MT348c-rNZYDWCJvzQ,39,0.6,"[(0.00977830773598, place), (0.00650128695031,..."
3,OTq55xUFMOz13DAi9D0_rw,uiJbTytfC1zsSOQ4aA3AWg,2014,12,23,4,"Good food, and good service",27,0,0,1,YfWmj8wWbuiCbnC6D8gvpw,48,0.9,"[(0.0146616328294, good), (0.0142812265826, fo..."
4,Kfbs3K37ZczrOQnoxgzNkw,II7cxU1cbOxVqzYY4ueflQ,2012,1,25,4,Came here today to get lunch to go. They were ...,661,1,0,2,kffxYQVQsEpF1DVlQlcGCg,5,0.9,"[(0.0063096357766, food), (0.00550394162605, g..."


In [241]:
reviews = pd.concat([train_data, test_data], axis=0)
reviews.reset_index(inplace=True, drop=True)
print reviews.describe()
reviews.head()

       topic_id_50  topic_prob_50
count      10000.0        10000.0
mean          27.1            0.5
std           14.7            0.2
min            0.0            0.0
25%           15.0            0.3
50%           28.0            0.5
75%           39.0            0.6
max           49.0            1.0


,business_id,review_id,date.year,date.month,date.day,stars,text,review_length,votes.cool,votes.funny,votes.useful,user_id,topic_id_50,topic_prob_50,word_dist_50
0,E7nk0lC317pGxMX56gvaVQ,onOLEBffcpWS_jcaBLCqdw,2008,6,17,1,This place is a huge waste of the space they m...,1239,3,4,2,eBFm-lABQiKpaUcPDfYOgA,39,0.6,"[(0.00977830773598, place), (0.00650128695031,..."
1,duHUQFn7K-Ybs1rPI4AOEQ,23z4mKkaMCXGFw4cm6xHxA,2012,4,15,3,I used to frequent Z-Tejas more frequently for...,936,1,1,2,y5_vrqIylhXMrejM_-x_vA,43,0.7,"[(0.0090865194881, good), (0.00526586990462, g..."
2,_K5cfCM7-zQZ1hXdbsc2fQ,dqrwXpcYdFvNYL5z1AeuZw,2013,12,1,3,"Meeting some friends for dinner, they live in ...",1119,0,0,0,8si1MT348c-rNZYDWCJvzQ,39,0.6,"[(0.00977830773598, place), (0.00650128695031,..."
3,OTq55xUFMOz13DAi9D0_rw,uiJbTytfC1zsSOQ4aA3AWg,2014,12,23,4,"Good food, and good service",27,0,0,1,YfWmj8wWbuiCbnC6D8gvpw,48,0.9,"[(0.0146616328294, good), (0.0142812265826, fo..."
4,Kfbs3K37ZczrOQnoxgzNkw,II7cxU1cbOxVqzYY4ueflQ,2012,1,25,4,Came here today to get lunch to go. They were ...,661,1,0,2,kffxYQVQsEpF1DVlQlcGCg,5,0.9,"[(0.0063096357766, food), (0.00550394162605, g..."


In [242]:
reviews.to_pickle('review/reviews_with_topics.pkl')

## Exploring LDA results

In [137]:
lda_topics_50

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
0,"(0.00610489308015, food)","(0.00536757561738, good)","(0.00407210558982, place)","(0.0037431312672, rice)","(0.00249762288012, indian)","(0.0023008090173, chicken)","(0.00226198641757, don)","(0.00219456855018, like)","(0.00216307033037, just)","(0.00199229067994, salsa)"
1,"(0.0129114067986, place)","(0.0116924222419, great)","(0.0112452136997, good)","(0.0107805161015, food)","(0.00670595031098, like)","(0.00641820631365, service)","(0.00501210798395, just)","(0.00483569448915, time)","(0.00436332886908, love)","(0.00377175229072, restaurant)"
2,"(0.00482024992253, great)","(0.00402117446831, place)","(0.00345826404804, dinner)","(0.00320387107332, night)","(0.00319201633009, pizza)","(0.00301513124258, like)","(0.00300217517098, time)","(0.00277097118748, just)","(0.00223578453725, don)","(0.00211370537121, service)"
3,"(0.0122478146418, great)","(0.00847685287049, good)","(0.00770774232001, place)","(0.00726859093747, really)","(0.00657945483174, food)","(0.00501285720165, friendly)","(0.00428646314986, best)","(0.00423643914798, delicious)","(0.00391213578462, definitely)","(0.00332780040856, staff)"
4,"(0.0127397694348, good)","(0.0110781049463, food)","(0.00828138620365, place)","(0.00800788520933, service)","(0.00576006623028, like)","(0.00575683043354, great)","(0.00503840480225, just)","(0.00442527083608, time)","(0.00394435622523, prices)","(0.00375750128566, lunch)"
5,"(0.00762185033754, food)","(0.00620193877476, thai)","(0.00426970215983, place)","(0.00424966945538, gluten)","(0.00408212640407, just)","(0.00348116702214, gluten free)","(0.00314722764889, free)","(0.00281950205348, lunch)","(0.002809281615, pizza)","(0.00260482400031, good)"
6,"(0.00376568444605, food)","(0.00360777920371, great)","(0.00328579866505, cracker)","(0.00324694230131, best)","(0.00283001269011, barrel)","(0.00282731617065, cracker barrel)","(0.00236391769028, amazing)","(0.00226081352857, place)","(0.00223443136003, amazing service)","(0.00221074391305, good)"
...,...,...,...,...,...,...,...,...,...,...
43,"(0.0090672103597, great)","(0.00826641387396, good)","(0.00773953300729, food)","(0.00770465873369, really)","(0.00412142279461, place)","(0.00365576156661, service)","(0.00356290503472, like)","(0.00331689573432, salad)","(0.00328418103968, chicken)","(0.0030116276693, ve)"
44,"(0.0040568933221, food)","(0.0038477261844, place)","(0.00334582186228, great)","(0.00307092778043, soup)","(0.00290089436506, location)","(0.00285722341969, good)","(0.00275068899664, like)","(0.00244766961488, cheese)","(0.00198826485981, just)","(0.00185837308806, best)"


In [123]:
top_10_words_50 = []
for index, row in lda_topics_df.iterrows():
    words = []
    for val in row:
        words.append(val[1])
    top_10_words_50.append(words)


In [124]:
topics_words_50 = pd.DataFrame(top_10_words_50, columns=lda_topics_50.columns)
topics_words_50


,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9
0,food,good,place,rice,indian,chicken,don,like,just,salsa
1,place,great,good,food,like,service,just,time,love,restaurant
2,great,place,dinner,night,pizza,like,time,just,don,service
3,great,good,place,really,food,friendly,best,delicious,definitely,staff
4,good,food,place,service,like,great,just,time,prices,lunch
5,food,thai,place,gluten,just,gluten free,free,lunch,pizza,good
6,food,great,cracker,best,barrel,cracker barrel,amazing,place,amazing service,good
...,...,...,...,...,...,...,...,...,...,...
43,great,good,food,really,place,service,like,salad,chicken,ve
44,food,place,great,soup,location,good,like,cheese,just,best


In [103]:
lda_model.show_topic(1)

[(0.01291140679858036, u'place'),
 (0.011692422241946748, u'great'),
 (0.01124521369970964, u'good'),
 (0.010780516101493305, u'food'),
 (0.0067059503109821172, u'like'),
 (0.0064182063136474796, u'service'),
 (0.0050121079839490733, u'just'),
 (0.0048356944891482104, u'time'),
 (0.0043633288690773479, u'love'),
 (0.0037717522907160629, u'restaurant')]

In [243]:
rest_reviews

,business_id,review_id,date.year,date.month,date.day,stars,text,review_length,votes.cool,votes.funny,votes.useful,user_id
0,9Ep4sguv3HH_8lWyzSogjw,owEJz2PJeyQiXELwDXgfqw,2009,8,18,4,Chomp chomp. Crunch crunch?\n\nWhy yes! That's...,1400,1,1,0,SIEb1KKCaH2CMrxt4azv1A
1,EHi-lebXGWjSktl4WvSoow,gkvyP2LoITViu8jXk5swgQ,2010,8,24,4,Jules was our server a few Wednesdays back for...,722,2,1,2,Q96IRvil6RNgdLmGKuh81A
2,1OA1jo0GlK9sf68f00w8ZQ,vla4f-IujCaYGkgIz6C01Q,2013,7,14,5,Great fast food place!! The prices can't be b...,218,0,0,0,-xPKyCJiK9q1OFZ7GCZwBw
3,KgTb63IZHFn_rhLG-cpm_A,h2RBvPYDFGCmxHX4PYNVAQ,2010,12,14,5,I heard from a friend that Renegade had become...,1682,2,0,3,oy6fdscGSXY2gzRqF9pZxg
4,IVKfggCQxQacIow9xP0fjA,I5abanoLK7QFkVkPiAvP9A,2013,12,11,4,Decide to try this place as I saw their menu a...,427,0,0,0,S3o_ekBuzmvnjFcTl_j6vg
5,VpHNYumY3F92a90AArkFyQ,5UkNQk8sCoBmQEVsagwA0g,2014,7,27,4,"For the price you pay, you get a decent amount...",243,0,0,0,uh8VYJuokQfjA-0CHUwQbg
6,fwYujhi514Ie5GsdIZsa2g,KRRlgdDwclxKI5V5qFQpTg,2014,10,2,5,They nailed it. Healthy food that tastes amazi...,1749,3,1,5,gzJpPaHN-NXBkAZcZri3hw
...,...,...,...,...,...,...,...,...,...,...,...,...
9993,svx5Gw-FJ6ZU9hb58aXgLg,HoeazfZb8QTRhDVqw83xKw,2014,8,18,5,This place never let's me down. Great food and...,172,0,0,1,_XKBfvm8QWOFJqKo87Ywrg
9994,jiWAo2z_seY5pQwhzrJiYA,w6gV9LYkdAxYQsKB2E_FTg,2013,9,29,4,Went in for just drinks w a large party. Staff...,479,0,0,0,Nsp_vUDdeinvlrTl1enHew


In [248]:
rest_reviews_all = pd.read_pickle('review/reviews300k.pkl')


# get restaurant reviews
train_data, test_data = train_test_split(rest_reviews_all, test_size=0.3)

In [249]:
train_data = pd.DataFrame(train_data, columns=rest_reviews_all.columns)
test_data = pd.DataFrame(test_data, columns=rest_reviews_all.columns)



In [251]:
vectorizer = CountVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2))
train_documents = train_data['text'].values
train_matrix = vectorizer.fit_transform(train_documents)
corpus = matutils.Sparse2Corpus(train_matrix.T)

In [ ]:
corpora.MmCorpus.serialize('/tmp/corpus300k.mm', corpus)

In [ ]:
corpus = corpora.MmCorpus('/tmp/corpus300k.mm')